In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("../Clean_Data/reddit_data.csv")
def cleanURL(url):
    url = url.replace("/"," ").replace("."," ").replace("_", " ").replace("-", " ")
    url = re.sub(r"[0-9]+", "", url)
    new_url = []
    for u in (url.split()):
        # print(u)
        if u != "reddit" and u != "comments" and u != "india" and u != "https:" and u != "http:" and len(u) > 4 and u != "www" and u != "com":
            new_url.append(u)
    return " ".join(new_url)

In [3]:
cleaned_urls = []
for t in data["url"]:
    cleaned_urls.append(cleanURL(t))

In [4]:
X = []
for i in range(2576):
    f = str(data["title"][i]) + str(data["comments"][i]) + str(cleaned_urls[i])
    X.append(f)
X = np.array(X)

In [5]:
MAX_WORDS = 10000
MAX_LEN = 200
EMBEDDING_DIM =50

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words = MAX_WORDS, lower=True)

Using TensorFlow backend.


In [7]:
tokenizer.fit_on_texts(X)

In [8]:
word_index = tokenizer.word_index

In [9]:
len(word_index)

70186

In [10]:
x = tokenizer.texts_to_sequences(X)
x = pad_sequences(x, maxlen=MAX_LEN)
print(x.shape)

(2576, 200)


In [11]:
y = pd.get_dummies(data['flair']).values

In [12]:
print(y.shape)

(2576, 12)


In [13]:
XT, Xt, YT, Yt = train_test_split(x,y,test_size=0.1,shuffle=True, random_state=42)

In [14]:
print(XT.shape, YT.shape)

(2318, 200) (2318, 12)


In [15]:
print(Xt.shape, Yt.shape)

(258, 200) (258, 12)


In [16]:
from keras.models import Sequential
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense

In [17]:
model = Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=x.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(12, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 50)           500000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 200, 50)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dense_1 (Dense)              (None, 12)                1212      
Total params: 561,612
Trainable params: 561,612
Non-trainable params: 0
_________________________________________________________________


In [18]:
epochs = 40
batch_size = 64

In [19]:
history = model.fit(XT, YT, epochs=epochs, batch_size=batch_size, validation_split=0.1)

Instructions for updating:
Use tf.cast instead.
Train on 2086 samples, validate on 232 samples
Epoch 1/40
2086/2086 [==============================] - 7s 3ms/step - loss: 2.4743 - acc: 0.1083 - val_loss: 2.4640 - val_acc: 0.0991
Epoch 2/40
2086/2086 [==============================] - 6s 3ms/step - loss: 2.4215 - acc: 0.1572 - val_loss: 2.4154 - val_acc: 0.1509
Epoch 3/40
2086/2086 [==============================] - 6s 3ms/step - loss: 2.2808 - acc: 0.2675 - val_loss: 2.2854 - val_acc: 0.1595
Epoch 4/40
2086/2086 [==============================] - 7s 3ms/step - loss: 1.9793 - acc: 0.3686 - val_loss: 2.1438 - val_acc: 0.2629
Epoch 5/40
2086/2086 [==============================] - 7s 3ms/step - loss: 1.6182 - acc: 0.5139 - val_loss: 1.9762 - val_acc: 0.3664
Epoch 6/40
2086/2086 [==============================] - 7s 3ms/step - loss: 1.2034 - acc: 0.6352 - val_loss: 1.9086 - val_acc: 0.4009
Epoch 7/40
2086/2086 [==============================] - 7s 3ms/step - loss: 0.9646 - acc: 0.7330 - va

In [20]:
accr = model.evaluate(Xt, Yt)

258/258 [==============================] - 0s 1ms/step


In [21]:
print("Accuracy : " , accr[1])

Accuracy :  0.42248062015503873
